In [ ]:
# Initialize the simulation parameters
from imports import *
from globalMethods import *
from vqteMethods import *
from exactDiagMethods import *
from fidelity_main import *

def main():
        # Initialize the simulation parameters
        run_multiple_layers(2,10, 0.05)
        
        gamma_L = 1
        gamma_R = 0

        eps = 1.0

        mu_L= 1.0
        mu_R= 1.0

        # Temperature of Reserviors
        T_L = 10
        T_R= 10

        time = 10
        dt = 0.05
        nt = int(time/dt)

        beta_L=1/T_L
        beta_R=1/T_R
        F_L = 1/(np.exp(beta_L*(eps-mu_L))+1)
        F_R = 1/(np.exp(beta_R*(eps-mu_R))+1)


        # Generate Hamiltonians
        exact_diag_ham = build_exact_diag_hamiltonian(eps)
        ham_real, ham_imag = hamiltonian_generation(eps, gamma_L, gamma_R, F_R,F_L)
        print("Exact Diag Ham: ", exact_diag_ham)
        print("Ham real: ", ham_real)
        print("Ham imag: ", ham_imag)


        # Generate initial states
        vqte_init_state, exact_diag_init_state, ansatz, init_param_values = build_initial_states(ham_real)
        print("Initil State : " , vqte_init_state)
        print("Exact Initial : ", exact_diag_init_state)

        # Perform exact diagonalization
        exact_diag_results, time_points = perform_exact_diag(gamma_L, F_L, gamma_R, F_R, dt, nt, exact_diag_init_state, exact_diag_ham)
    
        # Perform vqte simulation
        vqte_results = perform_vqte(ham_real, ham_imag, vqte_init_state, dt, nt, ansatz, init_param_values)

        #Output results
        output_results(vqte_results, exact_diag_results, time, nt, time_points)
        print(vqte_results[-1])
main()

Running simulation with 1 layers...
Number of layers 1
Step 0 out of 200
Step 1 out of 200
Step 2 out of 200
Step 3 out of 200
Step 4 out of 200
Step 5 out of 200
Step 6 out of 200
Step 7 out of 200
Step 8 out of 200
Step 9 out of 200
Step 10 out of 200
Step 11 out of 200
Step 12 out of 200
Step 13 out of 200
Step 14 out of 200
Step 15 out of 200
Step 16 out of 200
Step 17 out of 200
Step 18 out of 200
Step 19 out of 200
Step 20 out of 200
Step 21 out of 200
Step 22 out of 200
Step 23 out of 200
Step 24 out of 200
Step 25 out of 200
Step 26 out of 200
Step 27 out of 200
Step 28 out of 200
Step 29 out of 200
Step 30 out of 200
Step 31 out of 200
Step 32 out of 200
Step 33 out of 200
Step 34 out of 200
Step 35 out of 200
Step 36 out of 200
Step 37 out of 200
Step 38 out of 200
Step 39 out of 200


KeyboardInterrupt: 